In [202]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
%matplotlib tk

data = pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/doe_pickle_1458.pkl') # this one for average times

In [204]:
print(data.columns)

Index(['tm', 'dt', 'yy', 'mm', 'ww', 'wd', 'dd', 'xl', 'hh', 'an', 'vb', 'vl',
       'ht', 'tt', 'acw', 'bz', 'dectt', 'decht', 'decacw'],
      dtype='object')


In [205]:
# functions that will be apllied on grouping, left out dt bz and hh; dt and bz should be indices, hh can be dropped
colfunx_collapse_dt={'yy':'first', 'mm':'first', 'ww':'first', 'wd':'first', 'dd':'first', 'xl':'first', 'an':'sum', 'vb':'sum', 'vl':'sum', 'ht':'sum', 'tt':'sum', 'acw':'sum','decht':'sum', 'dectt':'sum', 'decacw':'sum'}
# group data by day and bearbzeit
data_grouped=data.groupby(['dt','bz']).agg(colfunx_collapse_dt)

In [206]:
# dates without neben- or kernzeit should contain each index, but fill them with null. Solution see https://stackoverflow.com/questions/17287933/filling-in-date-gaps-in-multiindex-pandas-dataframe
dg=data_grouped.unstack('dt').fillna(0).stack('dt')
dg2=dg.reorder_levels(['dt','bz'])

In [212]:
# averages as their own columns
dg2['aht']=(dg2['ht']/dg2['vb']).fillna(0)
dg2['att']=(dg2['tt']/dg2['vb']).fillna(0)
dg2['aacw']=(dg2['acw']/dg2['vb']).fillna(0)

# function for conversion of excel-floats to timedelta objects
def float_to_delta(xlfloat):
    deltatime=timedelta(xlfloat)
    return deltatime
dg2[['td_att','td_aht','td_aacw']] = dg2[['att', 'aht', 'aacw']].applymap(float_to_delta)

# reorder columns for better comprehensibility, left out the decimal times and the excel-averages
col_order=['xl','yy','mm','dd','ww','wd','an','vb','vl','ht','tt','acw','td_aht','td_att','td_aacw']
dg2=dg2[col_order]



In [213]:
kernzeit=dg2.xs('k', level=1, drop_level=False)
nebnzeit=dg2.xs('n', level=1, drop_level=False)
# both frames share the same index (level 0, i.e. datetime.date)

In [214]:
kernzeit.tail()

,,xl,yy,mm,dd,ww,wd,an,vb,vl,ht,tt,acw,td_aht,td_att,td_aacw
dt,bz,,,,,,,,,,,,,,,
2017-10-04,k,43012.0,2017.0,10.0,4.0,40.0,Wed,326.0,268.0,58.0,0.726447,0.600822,0.125625,00:03:54.197761,00:03:13.697761,00:00:40.500000
2017-10-05,k,43013.0,2017.0,10.0,5.0,40.0,Thu,303.0,257.0,46.0,0.739329,0.627847,0.111481,00:04:08.552529,00:03:31.073930,00:00:37.478599
2017-10-06,k,43014.0,2017.0,10.0,6.0,40.0,Fri,371.0,332.0,39.0,0.971354,0.815104,0.156250,00:04:12.786145,00:03:32.123494,00:00:40.662651
2017-10-07,k,43015.0,2017.0,10.0,7.0,40.0,Sat,66.0,50.0,16.0,0.167535,0.140706,0.026829,00:04:49.500000,00:04:03.140000,00:00:46.360000
2017-10-08,k,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0.000000,00:00:00,00:00:00,00:00:00
